In [ ]:
import time
import glob

In [ ]:
all_pdfs = glob.glob("docxs/*.pdf")

In [ ]:
all_data = {}

# PyPDF2

https://pypi.org/project/py2pdf/

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2

In [ ]:
for pdf_filename in all_pdfs:
    #print("=====",pdf_filename,"=====")
    pdf_file = open(pdf_filename, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    #print(number_of_pages,'pages\n')
    content = ""
    for indx in range(number_of_pages):
        page = read_pdf.getPage(indx)
        page_content = page.extractText()
        #print(page_content.encode('utf-8'))
        content += str(page_content.encode('utf-8'))
    try:
        all_data[pdf_filename]['pypdf2'] = content
    except KeyError:
        all_data[pdf_filename] = {'pypdf2': content}


# PDFminer

https://pypi.org/project/pdfminer/

https://github.com/pdfminer/pdfminer.six

In [ ]:
!pip install pdfminer.six

In [ ]:
for pdf_filename in all_pdfs:
    #print("=====",pdf_filename,"=====")
    !pdf2txt.py "$pdf_filename" > file_text.txt
    with open('file_text.txt','r') as file_handle:
        file_content = file_handle.read()
    all_data[pdf_filename]['pdfminer'] = file_content

# PDF to image to text (using tesseract)

In [ ]:
# used by pdf2image
!pip3 install Pillow # https://pypi.org/project/Pillow/

In [ ]:
# used by pdf2image
!conda install -c conda-forge poppler --yes

https://pypi.org/project/pdf2image/

https://pdf2image.readthedocs.io/en/latest/reference.html

In [ ]:
!pip3 install pdf2image

In [ ]:
!conda install -c conda-forge tesseract --yes

In [ ]:
!pip3 install pytesseract

In [ ]:
import pytesseract

In [ ]:
# depends on poppler being installed
from pdf2image import convert_from_path

In [ ]:
start_time = time.time()
for pdf_filename in all_pdfs:
    print(pdf_filename)
    pages = convert_from_path(pdf_filename, dpi=350)

    i = 1
    all_pages = ""
    for page in pages:
        print("page",i,round(time.time()-start_time,2),'seconds')
        image_name = pdf_filename + "_" + str(i) + ".jpg"  
        page.save(image_name, "JPEG")
        
        page_content = pytesseract.image_to_string(image_name, lang='eng')
        all_pages += page_content.lower().replace('.','')
    all_data[pdf_filename]['tesseract'] = all_pages
    

# comparison

In [ ]:
group_keywords = {"group 1": ["word1","word2"], # OR within that list
                  "group 2": ["word2", "corporation"], 
                  "group 3": ["bob"]}

In [ ]:
list_of_words = []
for group, word_list in group_keywords.items():
    for word in word_list:
        list_of_words.append(word)

In [ ]:
list_of_word_stems = list(set(list_of_words))

print all the content

show the matches per python library

In [ ]:
for filename, content_dict in all_data.items():
    #print(filename)
    list_of_matched_words = []
    for libraryname, content_str in content_dict.items():
        cleaned_str = content_str.lower().replace('.','')
        for this_word in list_of_word_stems:
            if this_word in cleaned_str: 
                list_of_matched_words.append(this_word)
    uniq_matches = list(set(list_of_matched_words))
    if len(uniq_matches)==0:
        print(len(uniq_matches),'keyword matches in',filename,'\n')
    else: # at least one match
        str_to_print = str(len(uniq_matches)) + ' keyword matches in ' + filename +': '
        for matched_word in uniq_matches:
            str_to_print += matched_word + ", "
        print(str_to_print)
        
        groups_to_alert = []
        for group, word_list in group_keywords.items():
            for word in word_list:
                if word in uniq_matches:
                    groups_to_alert.append(group)
        str_to_print = "alert "
        for group in list(set(groups_to_alert)):
            str_to_print += group + " and "
        print(str_to_print[:-4]+"\n")